In [ ]:
import re
import os
import shutil

from datetime import datetime

output_dir='./output_files/'

def snippet_filename(schema, version, counter):
    return '{}/{}-ddl-{}-{:02d}.sql'.format(schema, schema, version, counter)

default_stmt = ['--liquibase formatted sql\n\n']

def create_snippets(schema, order):
    start = r"CREATE TABLE IF NOT EXISTS `.*`.`(.*)`"
    end = r"\) ENGINE=InnoDB .*"
    
    fk = r"CONSTRAINT `(.*)` FOREIGN KEY \(`(.*)`\) REFERENCES (`(.*)`\.)?`(.*)` (.*)"

    file_name = '../dump/{}_ddl.sql'.format(schema)
    
    output_files=[]
    
    with open(file_name, encoding='UTF-8') as f:
        table_order = 1
        counter = 0
        
        all_lines = f.readlines()

        append=False
        stmt = default_stmt.copy()

        fk_stmt = []
        for line in all_lines:            
            m1 = re.search(start, line)
            m2 = re.search(end, line)
            m3 = re.search(fk, line)

            if (m1):
                append=True
                
                if len(stmt) == 0:
                    stmt = default_stmt.copy()

                table_name = m1.group(1)
                #print(table_name)
                
                changeset_comment='--changeset bibagimon@gmail.com:{}.{}\n'.format(order, table_order)
                stmt.append(changeset_comment)
                #print(changeset_comment)
                
                line = 'CREATE TABLE IF NOT EXISTS `{}`.`{}` (\n'.format(schema, table_name)

            if append and not m3:
                if m2:
                    line = line.replace('CHARSET=utf8 ', 'CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_520_ci ')
                    line = line.replace('CHARSET=utf8;', 'CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_520_ci;')
                    line = line.replace('CHARSET=latin1 ', 'CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_520_ci ')
                    line = line.replace('CHARSET=latin1;', 'CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_520_ci;')
                    
                    line = re.sub(r"AUTO_INCREMENT=\d+ ", "", line)
                    
                stmt.append(line)

            if m3:
                fk_id = m3.group(1)
                column_name = m3.group(2)
                ref_schema = m3.group(4)

                if not ref_schema:
                    ref_schema = schema

                ref_table = m3.group(5)
                last_part = m3.group(6)

#                 print(id, column_name, ref_schema, ref_table, last_part)
#                 print(line)
                fk_create = 'ALTER TABLE `{}`.`{}` ADD CONSTRAINT `{}` FOREIGN KEY (`{}`) REFERENCES `{}`.`{}` {};'.format(
                    schema, table_name, fk_id, column_name, ref_schema, ref_table, last_part)
                fk_drop = '--rollback ALTER TABLE `{}`.`{}` DROP FOREIGN KEY `{}`;'.format(
                    schema, table_name, fk_id)
                
                print(fk_create)
                print(fk_drop)

                # Fix the last line
                stmt[-1] = stmt[-1].replace(',', '')
            
                fk_stmt.append(fk_create)

            if m2:
                append=False
                stmt.append('--rollback DROP TABLE IF EXISTS `{}`.`{}`;\n'.format(schema, table_name))
                stmt.append('\n\n')
                
                table_order += 1
                                

        output_file=snippet_filename(schema, '1.0.0', 1)
        output_files.append(output_file)
        #print(output_file)

        if not os.path.exists(output_dir + schema):
            os.mkdir(output_dir + schema)

        with open(output_dir + output_file, 'w') as out:
            out.writelines(stmt)
            out.flush()
    return set(output_files)

In [ ]:
schemas = []
all_output_files=[]
for i, schema in enumerate(schemas):
    all_output_files.append(create_snippets(schema, i))

In [ ]:
for item in all_output_files:
    for file in sorted(item):
        print('<include file="{}" relativeToChangelogFile="true"/>'.format(file))